In [6]:
import numpy as np
#from scipy.misc import imread, imsave, imresize
from skimage.io import imread, imsave
from skimage.transform import resize as imresize
from scipy.spatial.distance import cosine
import os
import sys
import glob
import time
import pandas as pd
#sys.path.append('./utils/')
from utils.rgb_ind_convertor import *

In [7]:
baseDirectory = '/d2/studies/TF2DeepFloorplan/'
benchmark_path='/d2/studies/TF2DeepFloorplan/dataset/r3d_train.txt'
result_dir='/d2/studies/TF2DeepFloorplan/outJan19_TFR5_lr5e-4_ss25_RGB/0/' #change this to outdir of training
num_of_classes=11

In [8]:
def evaluateBounds_cosine(benchmark_path, result_dir, suffix=None, im_resize=True, gt_resize=True, train=False):
    """

    Parameters
    ----------
    benchmark_path : string
        Path to .txt file containing dataset paths.
    result_dir : string
        Directory containing result images to evaluate.
    suffix : string, optional
        Suffix to append to file name
    im_resize : boolean, optional
        Whether to resize result images to 512x512. The default is True.
    gt_resize : boolean, optional
        Whether to resize ground truth images to 512x512. The default is True.
    train : boolean, optional
        Whether to evaluate training data or test data. The default is False.

    Returns
    -------
    None.

    """
    gt_paths = open(benchmark_path, 'r').read().splitlines()
    d_paths = [p.split('\t')[2] for p in gt_paths] # 1 denote wall, 2 denote door, 3 denote room
    r_paths = [p.split('\t')[3] for p in gt_paths] # 1 denote wall, 2 denote door, 3 denote room
    cw_paths = [p.split('\t')[-1] for p in gt_paths] # 1 denote wall, 2 denote door, 3 denote room, last one denote close wall
    im_names = [p.split('/')[-1].split('.')[0] for p in gt_paths]
    im_paths = [os.path.join(result_dir, p.split('/')[-1].split('.')[0]).replace('_close', '_doors_windows') + '.png' for p in d_paths]
    data_dir = os.path.dirname(benchmark_path)
    sims=[]
    names = []
    for i in range(len(im_paths)):
        try:
            res_im  = imread(im_paths[i], pilmode='L')
            name = os.path.basename(im_paths[i]).split('_')[0]
            if train:
                gt_im = imread(os.path.join(data_dir, 'newyork/train/' + name + '_close.png'), pilmode='L')
            elif not train:
                gt_im = imread(os.path.join(data_dir, 'newyork/test/' + name + '_close.png'), pilmode='L')
                
            if im_resize:
                res_im = imresize(res_im, (512,512,3), mode='constant', cval=0, preserve_range=True)
            if gt_resize:
                gt_im = imresize(gt_im, (512,512,3), mode='constant', cval=0, preserve_range=True)
            res_im_1d = res_im.flatten()
            gt_im_1d = gt_im.flatten()
            res_im_1d = res_im_1d + 1e-6
            gt_im_1d = gt_im_1d + 1e-6
            cos = cosine(res_im_1d, gt_im_1d)
            sim = 1-cos
            print("Image " + str(name) + " similarity " + str(sim))
            sims.append(sim)
            names.append(name)
        except FileNotFoundError:
            pass
    df = pd.DataFrame([names, sims]).T
    df.columns=['image', 'similarity']
    df.sort_values(by='similarity', inplace=True, ascending=False)
    df.reset_index(drop=True, inplace=True)
    if suffix:
        df.to_csv('Cosine_EntranceSimilarity_Results_' + suffix + '.csv')
    elif not suffix:
        df.to_csv('Cosine_EntranceSimilarity_Results.csv')
    aveSim = np.mean(sims)
    print("Average cosine similarity " + str(aveSim))
    return df

In [9]:
def evaluateRooms_cosine(benchmark_path, result_dir, num_of_classes=11, suffix=None, im_resize=True, gt_resize=True, train=False):
    """

    Parameters
    ----------
    benchmark_path : string
        Path to .txt file containing dataset paths.
    result_dir : string
        Directory containing result images to evaluate.
    num_of_classes : int, optional
        Number of room classes. The default is 11.
    suffix : string, optional
        Suffix to append to file name.
    im_resize : boolean, optional
        Whether to resize result images to 512x512. The default is True.
    gt_resize : boolean, optional
        Whether to resize ground truth images to 512x512. The default is True.
    train : boolean, optional
        Whether to evaluate training data or test data. The default is False.

    Returns
    -------
    None.

    """
    gt_paths = open(benchmark_path, 'r').read().splitlines()
    r_paths = [p.split('\t')[3] for p in gt_paths] # 1 denote wall, 2 denote door, 3 denote room
    im_names = [p.split('/')[-1].split('.')[0] for p in gt_paths]
    im_paths = [os.path.join(result_dir, p.split('/')[-1].split('.')[0] + '_pred.png') for p in r_paths]
    data_dir = os.path.dirname(benchmark_path)
    sims=[]
    names = []
    for i in range(len(im_paths)):
        try:
            res_im  = imread(im_paths[i], pilmode='RGB')
            name = os.path.basename(im_paths[i]).split('_')[0]
            if train:
                gt_im = imread(os.path.join(data_dir, 'newyork/train/' + name + '_rooms.png'), pilmode='RGB')
                gt_im_ind = rgb2ind(gt_im, color_map=floorplan_fuse_map) 
            elif not train:
                gt_im = imread(os.path.join(data_dir, 'newyork/test/' + name + '_rooms.png'))
                gt_im_ind = rgb2ind(im, color_map=floorplan_fuse_map)
                
            if im_resize:
                res_im = imresize(res_im, (512,512,3), mode='constant', cval=0, preserve_range=True)
            if gt_resize:
                gt_im_ind = imresize(gt_im_ind, (512,512,3), mode='constant', cval=0, preserve_range=True)
            res_im_1d = res_im.flatten()
            gt_im_1d = gt_im_ind.flatten()
            res_im_1d = res_im_1d + 1e-6
            gt_im_1d = gt_im_1d + 1e-6
            cos = cosine(res_im_1d, gt_im_1d)
            sim = 1-cos
            print("Image " + str(name) + " similarity " + str(sim))
            sims.append(sim)
            names.append(name)
        except FileNotFoundError:
            pass
    df = pd.DataFrame([names, sims]).T
    df.columns=['image', 'similarity']
    df.sort_values(by='similarity', inplace=True, ascending=False)
    df.reset_index(drop=True, inplace=True)
    if suffix:  
        df.to_csv('Cosine_Similarity_Results_' + suffix + '.csv')
    elif not suffix:
            df.to_csv('Cosine_RoomSimilarity_Results.csv')

    aveSim = np.mean(sims)
    print("Average cosine similarity " + str(aveSim))
    return df


In [10]:
evaluateRooms_cosine(benchmark_path, result_dir, num_of_classes=11, im_resize=True, gt_resize=True, train=True)

Image 10 similarity 0.5529251395973739
Image 28906422 similarity 0.5667167534388347
Image 2 similarity 0.5920725351741191
Image 30044076 similarity 0.5882768291028723
Image 30049107 similarity 0.5872032914673244
Image 30957516 similarity 0.6139697337073904
Image 31272420 similarity 0.5931851329920934
Image 31318404 similarity 0.5532715247026951
Image 31431717 similarity 0.5455678135996771
Image 31483593 similarity 0.5801025962889311
Image 31491612 similarity 0.5851006971759909
Image 31566489 similarity 0.5830484180719426
Image 31573533 similarity 0.5683743192169463
Image 31677402 similarity 0.5728940320896595
Image 31683135 similarity 0.6033986413383925
Image 31727418 similarity 0.5877829745688238
Image 31814460 similarity 0.591910702495878
Image 31820961 similarity 0.5625201748569849
Image 31826949 similarity 0.6161679851492317
Image 31829949 similarity 0.594616915845239
Image 31830006 similarity 0.5749731649254262
Image 31830138 similarity 0.582882823941695
Image 31830141 similarity 

,image,similarity
0,45780106,0.644378
1,45764671,0.641144
2,31850325,0.630127
3,31826949,0.616168
4,46781618,0.615867
...,...,...
122,45708481,0.538354
123,31883331,0.529031
124,50,0.396829
125,39,0.334771


In [ ]:
evaluateBounds_cosine(benchmark_path, result_dir, suffix='Jan19_Epoch0', im_resize=True, gt_resize=True, train=True)